In [15]:
import os
from google.colab import drive
import training.train as train
from utils.ConfigLoader import ConfigLoader
from utils.hyperparameters import Hyperparameters

In [48]:
drive_dir = '/content/drive'
drive.mount(drive_dir)

dataset_dir = os.path.join(drive_dir,'MyDrive/data/mt')

model_hyperparameters = Hyperparameters(
    ConfigLoader("configs/config.yaml").get_config())

train.train(os.path.join(dataset_dir,
                        f"train7k_w{model_hyperparameters.window_size}.pt"),
            os.path.join(dataset_dir,
                        f"val7k_w{model_hyperparameters.window_size}.pt"),
            model_hyperparameters)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
training on cuda:0
Number of Batches: 2094
Batch Size: 200


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


total number of trainable parameters: 48375765
Layers:
source_embedding: Embedding(6797, 200)
target_embedding: Embedding(6461, 200)
fc_source: Linear(in_features=1000, out_features=300, bias=True)
fc_source_bn: BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
fc_target: Linear(in_features=400, out_features=300, bias=True)
fc_target_bn: BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
fc1: Linear(in_features=600, out_features=500, bias=True)
fc1_bn: BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
fc2: Linear(in_features=500, out_features=6461, bias=True)
fc2_bn: BatchNorm1d(6461, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
output_layer: Linear(in_features=6461, out_features=6461, bias=True)

        Starting Training: 

batch_accuracy:0.285
batch_perplexity:399.6766662597656
epoch: 0
steps: 10

batch_accuracy:0.265
batch_perplexity:290.364990234375
epoch: 0
steps: 20

ba

KeyboardInterrupt: 

In [50]:
from datetime import datetime
date = datetime.today().strftime('%d-%m-%Y')

In [51]:
!zip -r {dataset_dir}/checkpoints.zip eval/checkpoints/{date}/

updating: eval/checkpoints/12-06-2024/ (stored 0%)
updating: eval/checkpoints/12-06-2024/20_13_43.pth


zip error: Interrupted (aborting)


In [52]:
import torch
from assignments.assignment4 import *
from utils.file_manipulation import load_data
from preprocessing.dictionary import Dictionary

data_de = load_data("data/raw/multi30k.dev.de")
data_en = load_data("data/raw/multi30k.dev.en")

dict_de = Dictionary.load(os.path.join(dataset_dir, "dicts/train_dict_de_w2.pkl"))
dict_en = Dictionary.load(os.path.join(dataset_dir, "dicts/train_dict_en_w2.pkl"))

model = torch.load(f"eval/checkpoints/{date}/20_12_53.pth")

test_model_bleu(model, data_de, data_en, dict_de, dict_en,
                3, model_hyperparameters.window_size, True, None)

KeyboardInterrupt: 